In [35]:
import pandas as pd
import numpy as np
import typing as tp

from datetime import datetime, timedelta
from dateutil.parser import parse
from scipy.stats import levy_stable

In [18]:
# Import csv data for last 30 days of priceCumulative values from csv/data-1624665179.csv
df_raw = pd.read_csv("csv/data-1624665179.csv")
df_raw

,result,table,_start,_stop,_time,_value,_field,_measurement,id,influx-sushi,token0_name,token1_name,type
0,_result,0,2021-05-26 23:52:57.843291+00:00,2021-06-25 23:52:57.843291+00:00,2021-05-27 00:06:58+00:00,1.259956e+43,price1Cumulative,mem,Sushiswap: UNI / WETH,ingest-data-frame,UNI,WETH,metrics-hourly
1,_result,0,2021-05-26 23:52:57.843291+00:00,2021-06-25 23:52:57.843291+00:00,2021-05-27 00:13:42+00:00,1.259976e+43,price1Cumulative,mem,Sushiswap: UNI / WETH,ingest-data-frame,UNI,WETH,metrics-hourly
2,_result,0,2021-05-26 23:52:57.843291+00:00,2021-06-25 23:52:57.843291+00:00,2021-05-27 00:31:02+00:00,1.260029e+43,price1Cumulative,mem,Sushiswap: UNI / WETH,ingest-data-frame,UNI,WETH,metrics-hourly
3,_result,0,2021-05-26 23:52:57.843291+00:00,2021-06-25 23:52:57.843291+00:00,2021-05-27 00:31:40+00:00,1.260031e+43,price1Cumulative,mem,Sushiswap: UNI / WETH,ingest-data-frame,UNI,WETH,metrics-hourly
4,_result,0,2021-05-26 23:52:57.843291+00:00,2021-06-25 23:52:57.843291+00:00,2021-05-27 00:50:25+00:00,1.260089e+43,price1Cumulative,mem,Sushiswap: UNI / WETH,ingest-data-frame,UNI,WETH,metrics-hourly
...,...,...,...,...,...,...,...,...,...,...,...,...,...
81977,_result,29,2021-05-26 23:52:57.843291+00:00,2021-06-25 23:52:57.843291+00:00,2021-06-25 23:03:53+00:00,1.138973e+31,price1Cumulative,mem,Sushiswap: WETH / WBTC,ingest-data-frame,WETH,WBTC,metrics-hourly
81978,_result,29,2021-05-26 23:52:57.843291+00:00,2021-06-25 23:52:57.843291+00:00,2021-06-25 23:07:41+00:00,1.138973e+31,price1Cumulative,mem,Sushiswap: WETH / WBTC,ingest-data-frame,WETH,WBTC,metrics-hourly
81979,_result,29,2021-05-26 23:52:57.843291+00:00,2021-06-25 23:52:57.843291+00:00,2021-06-25 23:25:47+00:00,1.138976e+31,price1Cumulative,mem,Sushiswap: WETH / WBTC,ingest-data-frame,WETH,WBTC,metrics-hourly
81980,_result,29,2021-05-26 23:52:57.843291+00:00,2021-06-25 23:52:57.843291+00:00,2021-06-25 23:29:19+00:00,1.138977e+31,price1Cumulative,mem,Sushiswap: WETH / WBTC,ingest-data-frame,WETH,WBTC,metrics-hourly


In [19]:
# Filter for cols we care about: [id, _time, _field, _value] and sort by time
df = df_raw.filter(items=['id', '_time', '_field', '_value'])
df = df.sort_values(by='_time', ignore_index=True)
df

,id,_time,_field,_value
0,Sushiswap: COMP / WETH,2021-05-26 23:55:41+00:00,price0Cumulative,2.892970e+40
1,Sushiswap: COMP / WETH,2021-05-26 23:55:41+00:00,price1Cumulative,4.967069e+41
2,Sushiswap: MKR / WETH,2021-05-27 00:01:19+00:00,price0Cumulative,1.383139e+41
3,Sushiswap: MKR / WETH,2021-05-27 00:01:19+00:00,price1Cumulative,9.427158e+40
4,Sushiswap: SNX / WETH,2021-05-27 00:01:19+00:00,price0Cumulative,1.178501e+39
...,...,...,...,...
81977,Sushiswap: SUSHI / WETH,2021-06-25 23:49:11+00:00,price0Cumulative,6.736688e+38
81978,Sushiswap: RUNE / WETH,2021-06-25 23:49:11+00:00,price0Cumulative,2.776636e+38
81979,Sushiswap: WETH / DAI,2021-06-25 23:49:11+00:00,price1Cumulative,1.864844e+44
81980,Sushiswap: UNI / WETH,2021-06-25 23:49:12+00:00,price1Cumulative,1.401652e+43


In [20]:
# Examine stats for two pairs: WETH / DAI (less volatile) ...
df_weth_dai = df[(df['id'] == "Sushiswap: WETH / DAI") & (df['_field'] == "price1Cumulative")]
df_weth_dai

,id,_time,_field,_value
13,Sushiswap: WETH / DAI,2021-05-27 00:04:08+00:00,price1Cumulative,1.539463e+44
43,Sushiswap: WETH / DAI,2021-05-27 00:17:51+00:00,price1Cumulative,1.539585e+44
76,Sushiswap: WETH / DAI,2021-05-27 00:30:06+00:00,price1Cumulative,1.539693e+44
107,Sushiswap: WETH / DAI,2021-05-27 00:41:13+00:00,price1Cumulative,1.539790e+44
129,Sushiswap: WETH / DAI,2021-05-27 00:52:12+00:00,price1Cumulative,1.539886e+44
...,...,...,...,...
81881,Sushiswap: WETH / DAI,2021-06-25 23:03:57+00:00,price1Cumulative,1.864589e+44
81896,Sushiswap: WETH / DAI,2021-06-25 23:14:19+00:00,price1Cumulative,1.864647e+44
81924,Sushiswap: WETH / DAI,2021-06-25 23:23:57+00:00,price1Cumulative,1.864701e+44
81954,Sushiswap: WETH / DAI,2021-06-25 23:38:10+00:00,price1Cumulative,1.864782e+44


In [21]:
# ... and ALCX / WETH (more volatile)
df_alcx_weth = df[(df['id'] == "Sushiswap: ALCX / WETH") & (df['_field'] == "price1Cumulative")]
df_alcx_weth

,id,_time,_field,_value
15,Sushiswap: ALCX / WETH,2021-05-27 00:04:58+00:00,price1Cumulative,2.615926e+40
55,Sushiswap: ALCX / WETH,2021-05-27 00:20:01+00:00,price1Cumulative,2.616081e+40
69,Sushiswap: ALCX / WETH,2021-05-27 00:28:45+00:00,price1Cumulative,2.616171e+40
95,Sushiswap: ALCX / WETH,2021-05-27 00:37:25+00:00,price1Cumulative,2.616260e+40
123,Sushiswap: ALCX / WETH,2021-05-27 00:50:24+00:00,price1Cumulative,2.616393e+40
...,...,...,...,...
81873,Sushiswap: ALCX / WETH,2021-06-25 23:02:38+00:00,price1Cumulative,2.954404e+40
81892,Sushiswap: ALCX / WETH,2021-06-25 23:09:36+00:00,price1Cumulative,2.954439e+40
81926,Sushiswap: ALCX / WETH,2021-06-25 23:25:45+00:00,price1Cumulative,2.954520e+40
81943,Sushiswap: ALCX / WETH,2021-06-25 23:34:10+00:00,price1Cumulative,2.954562e+40


In [22]:
# ... and UNI / WETH (mid volatile)
df_uni_weth = df[(df['id'] == "Sushiswap: UNI / WETH") & (df['_field'] == "price0Cumulative")]
df_uni_weth

,id,_time,_field,_value
25,Sushiswap: UNI / WETH,2021-05-27 00:06:58+00:00,price0Cumulative,1.195348e+39
35,Sushiswap: UNI / WETH,2021-05-27 00:13:42+00:00,price0Cumulative,1.195369e+39
85,Sushiswap: UNI / WETH,2021-05-27 00:31:02+00:00,price0Cumulative,1.195424e+39
89,Sushiswap: UNI / WETH,2021-05-27 00:31:40+00:00,price0Cumulative,1.195426e+39
124,Sushiswap: UNI / WETH,2021-05-27 00:50:25+00:00,price0Cumulative,1.195484e+39
...,...,...,...,...
81629,Sushiswap: UNI / WETH,2021-06-25 20:54:49+00:00,price0Cumulative,1.323064e+39
81651,Sushiswap: UNI / WETH,2021-06-25 21:10:38+00:00,price0Cumulative,1.323107e+39
81773,Sushiswap: UNI / WETH,2021-06-25 22:16:09+00:00,price0Cumulative,1.323286e+39
81886,Sushiswap: UNI / WETH,2021-06-25 23:07:38+00:00,price0Cumulative,1.323427e+39


In [23]:
# TODO:
#  1. Compute the TWAP from PC value in df [x]
#  2. Fit Levy stable with scipy built in MLE code [x]
#  3. Report Levy stable params fit (a, b, mu, sig) [x]
#  4. Calc [e**(mu*t + sig * (t/a)**(1/a) * F^{-1}(1-alpha)) - 1] (VaR * d**m normalized for imbalance) for diff t vals
#  5. See how large/small C_p = 4. can be for various t, alpha combos

In [24]:
# Define some functions to calc twap
PC_RESOLUTION = 112

def compute_amount_out(twap_112: np.ndarray, amount_in: int) -> np.ndarray:
    rshift = np.vectorize(lambda x: int(x * amount_in) >> PC_RESOLUTION)
    return rshift(twap_112)

def get_twap(pc: pd.DataFrame, window: int, amount_in: int) -> pd.DataFrame:
    dp = pc.filter(items=['_value'])\
        .rolling(window=window)\
        .apply(lambda w: w[-1] - w[0], raw=True)
    
    # for time, need to map to timestamp first then apply delta
    dt = pc.filter(items=['_time'])\
        .applymap(parse)\
        .applymap(datetime.timestamp)\
        .rolling(window=window)\
        .apply(lambda w: w[-1] - w[0], raw=True)

    # with NaNs filtered out
    twap_112 = (dp['_value'] / dt['_time']).to_numpy()
    twap_112 = twap_112[np.logical_not(np.isnan(twap_112))]
    twaps = compute_amount_out(twap_112, amount_in)
    
    # window close timestamps
    t = pc.filter(items=['_time'])\
        .applymap(parse)\
        .applymap(datetime.timestamp)\
        .rolling(window=window)\
        .apply(lambda w: w[-1], raw=True)
    ts = t['_time'].to_numpy()
    ts = ts[np.logical_not(np.isnan(ts))]
    
    df = pd.DataFrame(data=[ts, twaps]).T
    df.columns = ['timestamp', 'twap']
    
    # filter out any twaps that are less than or equal to 0
    df = df[df['twap'] > 0]
    return df

In [25]:
# 1. Compute the TWAP from PC value in df
WINDOW = 6
UNIT_WETH = 1e18

df_weth_dai_twap = get_twap(pc=df_weth_dai, window=WINDOW, amount_in=UNIT_WETH)
df_alcx_weth_twap = get_twap(pc=df_alcx_weth, window=WINDOW, amount_in=UNIT_WETH)
df_uni_weth_twap = get_twap(pc=df_uni_weth, window=WINDOW, amount_in=UNIT_WETH)

In [26]:
df_weth_dai_twap

,timestamp,twap
0,1.62208e+09,2823283256860140896256
1,1.62208e+09,2807416500389253480448
2,1.62208e+09,2795741643020610568192
3,1.62208e+09,2778509477732340465664
4,1.62208e+09,2767564848768069140480
...,...,...
3744,1.62466e+09,1832467229514366976000
3745,1.62466e+09,1825074114325430403072
3746,1.62466e+09,1817085838281425027072
3747,1.62466e+09,1810494773684469760000


In [27]:
df_weth_dai_twap['twap'] / 1e18

0       2823.28
1       2807.42
2       2795.74
3       2778.51
4       2767.56
         ...   
3744    1832.47
3745    1825.07
3746    1817.09
3747    1810.49
3748     1810.8
Name: twap, Length: 3749, dtype: object

In [41]:
# Define some functions to fit TWAP to log stable
def get_rs(twap: pd.DataFrame) -> np.ndarray:
    sample = twap['twap'].to_numpy()
    return [
        np.log(sample[i]/sample[i-1])
        for i in range(1, len(sample), 1)
    ]

def fit_to_log_stable(twap: pd.DataFrame) -> tp.Tuple[float]:
    rs = get_rs(twap)
    return levy_stable.fit(rs)

def get_params(twap: pd.DataFrame, period: int) -> pd.DataFrame:
    a, b, loc, scale = fit_to_log_stable(twap)

    # transform to mu, sig
    mu = loc / period
    sig = scale / (period/a)**(1/a)
    
    df = pd.DataFrame(data=[a, b, mu, sig]).T
    df.columns = ['a', 'b', 'mu', 'sig']
    return df

In [ ]:
# 2. Fit Levy stable with scipy built in MLE code
PERIOD = 600 # 10 min

# NOTE: this is taking too long ... what's a way around this while still using scipy?
# Could wrap C++ code from Nolan instead ...
df_weth_dai_params = get_params(twap=df_weth_dai_twap, period=PERIOD)
df_alcx_weth_params = get_params(twap=df_alcx_weth_twap, period=PERIOD)
df_uni_weth_params = get_params(twap=df_uni_weth_twap, period=PERIOD)

In [ ]:
# 3. Report Levy stable params fit (a, b, mu, sig)
df_weth_dai_params